# Selective Logging Detection with Deep Learning and GEE

## Exportación de Datos

## Contenido

1. [Definiciones y contexto](#1.-Definiciones-y-contexto)
2. [Exploración de datos](#2.-Exploración-de-datos)
3. [Procesamiento de la imagen](#3.-Procesamiento-de-la-imagen)  
    3.1 [Índices espectrales](#3.1-Índices-espectrales)  
    3.2 [Escalamiento (Normalización)](#3.2-Escalamiento-(Normalización))  
    3.3 [Asignar un valor numérico a la clase](#3.3-Asignar-un-valor-numérico-a-la-clase)  
    3.4 [Crear la banda "clase" de la imagen](#3.4-Crear-la-banda-"clase"-de-la-imagen)  
    3.5 [Transformación de la imagen ](#3.5-Transformación-de-la-imagen)  
4. [Exportación de los datos](#4.-Exportación-de-los-datos)

---

## 1. Definiciones y contexto

La tala selectiva es la cosecha de **árboles maderables valiosos**. Detectar esta actividad es complicado, debido a la naturaleza de ésta. Ya que, a diferencia de la deforestación, las áreas de pérdida de bosque dejadas por esta actividad muchas veces no son detectables usando los sensores tradicionales de resolución media, como Landsat o Sentinel-2. 

Es por ello que las imágenes satelitales de muy alta resolución como Skysat (0.5 m) nos permiten detectar con mayor exactitud esta actividad.

<img src="./pictures/Tala_A.jpg" width=800 alt="Imagen de tala">

La tala selectiva muchas veces se encuentra asociada a **infraestructuras**, como caminos forestales, patios de acopio de trozas de madera, campamentos madereros, entre otros. La identificación de estas infraestructuras facilita la detección de las actividades de tala selectiva.

<img src="./pictures/Tala_B.jpg" width=800 alt="Imagen de infraestructuras">

Algunas de las infraestructuras asociadas a la tala selectiva son sencillas de detectar, como los caminos forestales, sin embargo otras son más complejas, como los campamentos forestales.

[Volver al contenido](#Contenido)

---

## 2. Exploración de datos

Primero importaremos la [librería oficial](https://developers.google.com/earth-engine/guides/python_install) de Earth Engine en **Python** para la visualización de datos y su procesamiento utilizando `import`.

In [4]:
import ee

Debido a que el API EarthEngine requiere de una cuenta personal, procederemos a autenticar nuestra cuenta y a inicializar EarthEngine.  
Al correr la siguiente celda aparecerá un enlace, donde debemos generar un token y permitir a EarthEngine a que acceda a nuestro notebook.  
Debemos copiar y pegar el código de autorización.

In [5]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AWtgzh5_58TCyA3WcLIcUvV4uoXtSCv2w3rvT_4_4SLviHeVuXUKjuYxam0

Successfully saved authorization token.


Debemos verificar también si poseemos acceso al Google Cloud Storage. Con la siguiente función autorizamos a Google Colab a que acceda a la nube de Google.

In [ ]:
from google.colab import auth
auth.authenticate_user()

Definiremos por defecto nuestro proyecto y veremos la lista de elementos dentro.

In [96]:
project_id = 'deeplearning-306718'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].


gs://logging-dataset/
gs://logging-roads-dataset/
gs://mining-infrastructure-dataset/
gs://modis-suomi/
gs://nicfi/
gs://rtc-asf/
gs://s5p/
gs://skysat-acca/


Por último importaremos otra librería importante llamada [Geemap](https://geemap.org/), que nos permitirá visualizar los datos utilizando [folium](https://python-visualization.github.io/folium/).  En caso no tenemos **geemap**, podemos descargarlo usando el comando `!pip install`

In [11]:
#!pip install geemap
import geemap.foliumap as geemap

Utilizando el API de Earth Engine podemos visualizar los datos utilizando python y geemap. Cargaremos una imagen satelital de SkySat de muy alta resolución (0.5 m), así como los datos de entrenamiento recolectados.

In [61]:
img = ee.Image('projects/ACCA-SERVIR/Colaboraton/Skysat_Fatima')
pols = ee.FeatureCollection('projects/ACCA-SERVIR/Colaboraton/Skysat_Fatima_pols')

In [62]:
Map = geemap.Map(center = (-9.653, -74.198), zoom = 15)
Map.addLayer(img, {'bands': ['b3', 'b2', 'b1'], 'min': 3000, 'max': 12000}, 'SkySat')
Map.addLayer(pols, {'color': 'red'}, 'Data')
Map

Cantidad de datos (polígonos de tala):

In [63]:
print(pols.size().getInfo())

246


[Volver al contenido](#Contenido)

---

## 3. Procesamiento de la imagen

## 3.1 Índices espectrales

Los índices espectrales permiten resaltar la sensitividad de píxeles, a través de operaciones algebraicas entre bandas.  
Son muy útiles para acelerar el entrenamiento y permitir que el modelo generalize mejor.  
En este ejercicio se creará el índice NDVI y se añadirá a la imagen SkySat.

In [64]:
def addNDVI(img):
    ndvi = img.expression(
        '(NIR - RED) / (NIR + RED)', {
        'NIR': img.select('b4'),
        'RED': img.select('b3')
        }
    ).rename('ndvi')
    return img.addBands(ndvi)

In [65]:
img = addNDVI(img)

Utilizando geemap podemos ver el índice NDVI ya creado

In [66]:
Map = geemap.Map(center = (-9.653, -74.198), zoom = 15)
Map.addLayer(img, {'bands': ['ndvi'], 'palette': ['blue', 'yellow', 'green'], 'min': -0.8, 'max': 0.8}, 'NDVI')
Map.addLayer(pols, {'color': 'red'}, 'Data')
Map

## 3.2 Escalamiento (Normalización)

Para Deep Learning se recomienda escalar los datos, sobretodo cuando están sometidos a diferentes condiciones, en este ejercicio se realizará una **normalización** (escalamiento) de datos para que los valores pixelares estén de 0 a 1.  
Para ello utilizaremos la siguiente función:

In [67]:
def normalizationImg (img, bands, min, max, geo):
    desMin = ee.Number(min)
    desMax = ee.Number(max)
    strMin = ee.String('_min')
    strMax = ee.String('_max')
    def bandNorm (band):
        selectedBand = ee.Image(img).select(band)
        bandStats = selectedBand.reduceRegion(**{
            'reducer': ee.Reducer.minMax(),
            'geometry': geo,
            'scale': 0.5,
            'crs': 'EPSG:32719',
            'maxPixels': 1e12
            })
        bandString = ee.String(band)
        bandMinString = (bandString.cat(strMin))
        bandMaxString = (bandString.cat(strMax))
        bandMin = ee.Number(bandStats.get(bandMinString))
        bandMax = ee.Number(bandStats.get(bandMaxString))
        bandNormalized = ((selectedBand.subtract(bandMin).multiply(desMax.subtract(desMin).divide(bandMax.subtract(bandMin))).add(desMin)).rename(bandString.cat(ee.String('_normalized'))))
        return bandNormalized.toFloat()
    for b in bands:
        img = img.addBands(bandNorm(b))
         
    return img

Esta función escala los valores al valor mínimo y máximo deseado, permitiendo seleccionar las bandas a procesar

In [68]:
img_norm = normalizationImg(img, ['b1', 'b2', 'b3', 'b4'], 0, 1, img.geometry())

Veremos que la función añade bandas normalizadas adicionales, las cuales servirán para exportar los datos.  
Nótese que no se normaliza el índice NDVI, debido a que ya se encuentra _normalizado_.

In [69]:
print(img_norm.bandNames().getInfo())

['b1', 'b2', 'b3', 'b4', 'ndvi', 'b1_normalized', 'b2_normalized', 'b3_normalized', 'b4_normalized']


## 3.3 Asignar un valor numérico a la clase

Nuestra base de datos consiste en polígonos, que presentan un atributo llamado **tipo**, el cual no es numérico, por lo que crearemos un atributo con el valor de 1, para representar numéricamente los casos de tala.  
Podemos observar los valores del primer polígono.

In [70]:
print(pols.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-74.20131834249987, -9.646241075424708], [-74.20131395355791, -9.646245561421832], [-74.20130938547537, -9.646245677105858], [-74.20130490696295, -9.646245714675706], [-74.20130042845042, -9.64624575224548], [-74.20129586036754, -9.646245867929274], [-74.20129138185479, -9.6462459054989], [-74.20128681377167, -9.646241494641858], [-74.20128224568846, -9.64623708378515], [-74.20128215611818, -9.64623263535913], [-74.20128663463116, -9.646228071249496], [-74.20128654506091, -9.646223622823896], [-74.20129102357377, -9.646223585253951], [-74.20130007016941, -9.646223431999763], [-74.20130454868195, -9.646223394429596], [-74.20130911676462, -9.646223278745268], [-74.20131816335939, -9.646227652030708], [-74.20131825292962, -9.646232100457091], [-74.20131834249987, -9.646241075424708]]]}, 'id': '00000000000000000000', 'properties': {'id': 24, 'imgID': 'Skysat_20201017_ssc2_150055', 'tipo': 'tala', 'type': 'Polygon'}}


Utilizando la siguiente función, mapeamos cada polígono de nuestro conjunto de datos para asignarle el atributo **class** el valor de 1.

In [71]:
def add1(f):
    return f.set('class', 1.0)

Podemos observar la presencia del nuevo atributo creado.

In [72]:
pols = pols.map(add1)
print(pols.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-74.20131834249987, -9.646241075424708], [-74.20131395355791, -9.646245561421832], [-74.20130938547537, -9.646245677105858], [-74.20130490696295, -9.646245714675706], [-74.20130042845042, -9.64624575224548], [-74.20129586036754, -9.646245867929274], [-74.20129138185479, -9.6462459054989], [-74.20128681377167, -9.646241494641858], [-74.20128224568846, -9.64623708378515], [-74.20128215611818, -9.64623263535913], [-74.20128663463116, -9.646228071249496], [-74.20128654506091, -9.646223622823896], [-74.20129102357377, -9.646223585253951], [-74.20130007016941, -9.646223431999763], [-74.20130454868195, -9.646223394429596], [-74.20130911676462, -9.646223278745268], [-74.20131816335939, -9.646227652030708], [-74.20131825292962, -9.646232100457091], [-74.20131834249987, -9.646241075424708]]]}, 'id': '00000000000000000000', 'properties': {'class': 1, 'id': 24, 'imgID': 'Skysat_20201017_ssc2_150055', 'tipo': 'tala', 'type': 'Pol

## 3.4 Crear la banda "clase" de la imagen 

El siguiente paso, debido a que queremos exportar objetos y no pixeles individuales, es crear una banda adicional a la imagen, cuyo valor sea 1 para aquellos pixeles en su interior, y 0 para aquellos en el exterior.  
Para ello utilizaremos la siguiente función:

In [73]:
def createLabel (feature):
    featImg = feature.reduceToImage(['class'], ee.Reducer.mean())
    return featImg.toFloat().rename('class')

Aplicamos la función y añadimos la banda creada a la imagen normalizada.

In [74]:
img_norm = img_norm.addBands(createLabel(pols))

Nuevamente, obtenemos las bandas de la imagen. Nótese la existencia de la banda **class**, que vendrá a ser la clase, en este caso tala.

In [75]:
print(img_norm.bandNames().getInfo())

['b1', 'b2', 'b3', 'b4', 'ndvi', 'b1_normalized', 'b2_normalized', 'b3_normalized', 'b4_normalized', 'class']


## 3.5 Transformación de la imagen 

Se realizará una transformación de la imagen, en la que cada pixel será una matriz de los píxeles vecinos.  
Esto permite que al exportar puntos, en realidad se estén exportando imágenes (tensores en este caso).  
Para ello, debemos crear un kernel. Que será la repetición de listas, con el valor de 1.

In [76]:
row = ee.List.repeat(1, 64)
rows = ee.List.repeat(row, 64)
kernel = ee.Kernel.fixed(64, 64, rows)

Podemos visualizar al kernel creado. Que tiene un tamaño de 64 pixeles de ancho, y 64 pixeles de alto.

In [77]:
print(kernel.getInfo())

{'type': 'Kernel.fixed', 'width': 64, 'height': 64, 'weights': '\n  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]\n  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]\n  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

Utilizando el kernel, podremos realizar la transformación de la imagen.

In [78]:
img_final = img_norm.neighborhoodToArray(kernel, 0)

Finalmente seleccionaremos las variables predictoras, que formarán parte de la exportación y entrenamiento.

In [89]:
featureNames = ['b1_normalized', 'b2_normalized', 'b3_normalized', 'b4_normalized', 'ndvi', 'class']

In [80]:
img_final = img_final.select(featureNames)

In [81]:
print(img_final.bandNames().getInfo())

['b1_normalized', 'b2_normalized', 'b3_normalized', 'b4_normalized', 'ndvi', 'class']


[Volver al contenido](#Contenido)

---

## 4. Exportación de datos

Antes de realizar la exportación, se deben definir aquellos polígonos que se utilizarán como entrenamiento (70%), validación (20%) y prueba (10%).  
Para ello se utiliza la función `randomColumn()`

In [82]:
pols_final = pols.randomColumn()
print(pols_final.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-74.20131834249987, -9.646241075424708], [-74.20131395355791, -9.646245561421832], [-74.20130938547537, -9.646245677105858], [-74.20130490696295, -9.646245714675706], [-74.20130042845042, -9.64624575224548], [-74.20129586036754, -9.646245867929274], [-74.20129138185479, -9.6462459054989], [-74.20128681377167, -9.646241494641858], [-74.20128224568846, -9.64623708378515], [-74.20128215611818, -9.64623263535913], [-74.20128663463116, -9.646228071249496], [-74.20128654506091, -9.646223622823896], [-74.20129102357377, -9.646223585253951], [-74.20130007016941, -9.646223431999763], [-74.20130454868195, -9.646223394429596], [-74.20130911676462, -9.646223278745268], [-74.20131816335939, -9.646227652030708], [-74.20131825292962, -9.646232100457091], [-74.20131834249987, -9.646241075424708]]]}, 'id': '00000000000000000000', 'properties': {'class': 1, 'id': 24, 'imgID': 'Skysat_20201017_ssc2_150055', 'random': 0.1643305761441248

Podemos observar que el primer polígonos ahora tiene un atributo adicional llamado "Random", el cual servirá para definir aquellos datos de entrenamiento, validación y prueba.  
Utilizando el valor del atributo creado, dividiremos el conjunto de datos.

In [83]:
training_pols = pols_final.filter(ee.Filter.lt('random', 0.7))
validation_pols = pols_final.filter(ee.Filter.gte('random', 0.7)).filter(ee.Filter.lt('random', 0.9))
testing_pols = pols_final.filter(ee.Filter.gte('random', 0.9))

Podemos verificar la cantidad de polígonos de entrenamiento, validación y prueba.

In [84]:
print(training_pols.size().getInfo())
print(validation_pols.size().getInfo())
print(testing_pols.size().getInfo())

168
48
30


Crearemos listas en las que se incluirán los polígonos.

In [86]:
trainingList = training_pols.toList(training_pols.size())
validationList = validation_pols.toList(validation_pols.size())
testingList = testing_pols.toList(testing_pols.size())

Por último, se realiza la exportación de los datos al **Google Cloud Platform**  (también se pueden exportar a Google Drive o a un asset). Se utilizará un bucle para exportar cada polígono en cada archivo por separado. Utilizaremos la función `for` para ello.

In [95]:
for i in range(training_pols.size().getInfo()):
    sample = img_final.sample(
        region = ee.Feature(trainingList.get(i)).geometry(),
        scale = 0.5,
        numPixels = 1,
        seed = i,
        tileScale = 8
    )
    trainingTask = ee.batch.Export.table.toCloudStorage(
        collection = sample,
        description = 'train_' + str(i),
        bucket = 'colaboraton-logging',
        fileNamePrefix = 'train_' + str(i),
        fileFormat = 'TFRecord',
        selectors = featureNames
    )
    trainingTask.start()
print('Datos de entrenamiento exportados')

for i in range(validation_pols.size().getInfo()):
    sample = img_final.sample(
        region = ee.Feature(validationList.get(i)).geometry(),
        scale = 0.5,
        numPixels = 1,
        seed = i,
        tileScale = 8
    )
    validationTask = ee.batch.Export.table.toCloudStorage(
        collection = sample,
        description = 'validation_' + str(i),
        bucket = 'colaboraton-logging',
        fileNamePrefix = 'validation_' + str(i),
        fileFormat = 'TFRecord',
        selectors = featureNames
    )
    validationTask.start()
print('Datos de validación exportados')

for i in range(testing_pols.size().getInfo()):
    sample = img_final.sample(
        region = ee.Feature(testingList.get(i)).geometry(),
        scale = 0.5,
        numPixels = 1,
        seed = i,
        tileScale = 8
    )
    testingTask = ee.batch.Export.table.toCloudStorage(
        collection = sample,
        description = 'testing_' + str(i),
        bucket = 'colaboraton-logging',
        fileNamePrefix = 'testing_' + str(i),
        fileFormat = 'TFRecord',
        selectors = featureNames
    )
    testingTask.start()
print('Datos de prueba exportados')

Datos de validación exportados
Datos de prueba exportados


Procederemos a verificar el estado de exportación de los archivos en nuestro bucket del **Google Cloud Storage**

In [98]:
!gsutil ls gs://colaboraton-logging/*.gz

gs://rtc-asf/train_0.tfrecord.gz
gs://rtc-asf/train_1.tfrecord.gz
gs://rtc-asf/train_10.tfrecord.gz
gs://rtc-asf/train_100.tfrecord.gz
gs://rtc-asf/train_101.tfrecord.gz
gs://rtc-asf/train_102.tfrecord.gz
gs://rtc-asf/train_11.tfrecord.gz
gs://rtc-asf/train_12.tfrecord.gz
gs://rtc-asf/train_13.tfrecord.gz
gs://rtc-asf/train_14.tfrecord.gz
gs://rtc-asf/train_15.tfrecord.gz
gs://rtc-asf/train_16.tfrecord.gz
gs://rtc-asf/train_17.tfrecord.gz
gs://rtc-asf/train_18.tfrecord.gz
gs://rtc-asf/train_19.tfrecord.gz
gs://rtc-asf/train_2.tfrecord.gz
gs://rtc-asf/train_20.tfrecord.gz
gs://rtc-asf/train_21.tfrecord.gz
gs://rtc-asf/train_22.tfrecord.gz
gs://rtc-asf/train_23.tfrecord.gz
gs://rtc-asf/train_24.tfrecord.gz
gs://rtc-asf/train_25.tfrecord.gz
gs://rtc-asf/train_26.tfrecord.gz
gs://rtc-asf/train_27.tfrecord.gz
gs://rtc-asf/train_28.tfrecord.gz
gs://rtc-asf/train_29.tfrecord.gz
gs://rtc-asf/train_3.tfrecord.gz
gs://rtc-asf/train_30.tfrecord.gz
gs://rtc-asf/train_31.tfrecord.gz
gs://rtc-asf/tr

[Volver al contenido](#Contenido)

---